In [6]:
import pandas as pd
import json
import requests

In [7]:
#First I read in the data
df = pd.read_csv("Campaign_Contributions.csv")

#Since I know I will be interested in value amounts, and not just strings,
#I go ahead and convert the 'Amount' column to float.
df['Amount'] = df['Amount'].replace( '[\$,)]','', regex=True ).astype(float)

#Then I convert the date to real datetime, so I can select specific periods of time
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.sort_index(axis=0, inplace=True)

# How do I decide what campaign donations to pay attention to?

## Here are some ideas:
* Donations since Cuomo's official (announced) candidacy on 11/15/2016
* Donations made to "Andrew Cuomo 2018, Inc."
* Donations in the last 4 years before November 2018
* Total donations to Cuomo since he has been governor.
* Donations to Cuomo since he has been a politician (Attorney General)

In [8]:
#Andrew Cuomo announced his official candidacy on November 15, 2016
#I could just look at those:

df['2017-11-15':]['Amount'].sum()

4056770.0

In [9]:
#Or I could look at all donations made to "Andrew Cuomo 2018, Inc."... that yields a wonky sum of donations:

df[(df['Recipient']=='ANDREW CUOMO 2018 INC.')]['Amount'].sum()

112922258.0

In [10]:
#Or I could look at the last 4 years before November 2018.
#That seems to get me the closest to the "31" number that everyone (including the NYT) is citing.

df['2014-11-01':]['Amount'].sum()

30078564.0

In [11]:
#I'll decide to go with the last 4 years before November 2018, but can change this later easily.
df_warchest = df['2014-11-01':]

In [12]:
#So here is our Cuomo 2018 warchest data. 5244 campaign contributions from 2014-11-01 to 2018-01-12.
#The next BOE data update will be in July 2018.
#Some basic information about the columns:

df_warchest.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5244 entries, 2014-11-01 to 2018-01-12
Data columns (total 10 columns):
Donor                5243 non-null object
Address              5244 non-null object
Filer ID             5244 non-null object
Check No.            5138 non-null object
Amount               5244 non-null float64
Recipient            5244 non-null object
Transaction Code     5244 non-null object
Contribution Code    3326 non-null object
Contribution Type    48 non-null object
Fiscal Report        5244 non-null object
dtypes: float64(1), object(9)
memory usage: 450.7+ KB


In [13]:
#So let's take a look. Cuomo definitely prefers bigger donors, and less of them. The average donation is $5735 (!)
df_warchest['Amount'].describe()

count     5244.000000
mean      5735.805492
std       9304.667863
min      -1000.000000
25%        500.000000
50%       2000.000000
75%       5000.000000
max      87754.000000
Name: Amount, dtype: float64

In [14]:
#Who are Cuomo's top 25 donors, by number of times they have donated (in the last 4 years)?

df_warchest['Donor'].value_counts().head(25).to_frame()

,Donor
SANDRA WILKIN,9
PHILLIPS LYTLE LLP,9
JP MORGAN CHASE BANK,8
WELLS FARGO,8
SIGNATURE BANK,8
STERLING NATIONAL BANK,8
TRANSPORT WORKERS UNION LOCAL 100 PAC,8
JOSEPH JULIANO,8
DRIVE COMMITTEE POLITICAL FUND - TEAMSTERS,7
RICHARD OSTROFF,7


In [69]:
#Who are Cuomo's top individual donors, by dollar amounts they have given?
df_warchest.sort_values(by='Amount',ascending=False)[['Donor','Amount']].head(50)

,Donor,Amount
Date,,
2017-12-31,SIGNATURE BANK,87754.0
2017-07-19,JOSEPH DIMENNA,65100.0
2017-01-06,FISHER BROTHERS FINANCIAL AND DEVELOPMENT CO LLC,65000.0
2016-07-14,RICHARD MERKIN,65000.0
2017-05-19,GABELI HOLDINGS LLC,65000.0
2017-01-06,FISHER BROTHERS MANAGEMENT CO LLC,65000.0
2017-03-13,RAVENEL CURRY III,65000.0
2017-01-16,NEW YORK CAMPAIGN FOR ACHIEVEMENT NOW,65000.0
2017-12-21,NATHANIEL SIMONS,65000.0


# Next step: Take these, and query the OpenCorporates API to get the business names of all 3684 unique addresses.

## Plan:

Working with this data may be more like a war of attrition, with various passes matching the addresses or donor names to companies, 1-to-1.

Passes:

1. NYC-DB -- since I know real estate is such a significant portion of investors in Cuomo, start with NYCDB.
2. Officers -- Find those donor names matching officer names registered with OpenCorporates.
    a. What about common names -- should I limit jurisdiction to New York?
3. Registered addresses -- Find those addresses that match with registered addresses on OpenCorporates

In [16]:
df_warchest['Address'].nunique()

3684

In [51]:
def get_company_names(row):
    #If the contribution code is 'Individual', check for officer name!
    if row['Contribution Code'] == 'Individual':
        donor = str(row['Donor']).replace(",","+").replace(" ","+")
        base_url= "https://api.opencorporates.com/v0.4/officers/search?q="
        api_token= "&api_token=BSSwhXpP7xJ5couOddoY"
        response = requests.get(base_url+donor+api_token)
        print(base_url+donor+api_token)
        data = json.loads(response.content)
        if len(data['results'])>0:
            for officer in data['results']['officers']:
                print("Officer name:", officer['officer']['name'])
                print("Officer address:", officer['officer']['address'])
                print("Officer company name:", officer['officer']['company']['name'],"\n")
    
    else:
        address = "&registered_address=" + str(row['Address'].replace(",","+").replace(" ","+"))
        base_url= "https://api.opencorporates.com/v0.4/companies/search?q="
        api_token= "&api_token=BSSwhXpP7xJ5couOddoY"
        response = requests.get(base_url+address+api_token)
        data = json.loads(response.content)
        if len(data['results']['companies'])>0:
            print("Original Address and Donor: ", row['Address'], row['Donor'])
            count = 1
            for company in data['results']['companies']:
                if not company['company']['inactive']:
                    print("Matched Registered Address", count)
                    print("Company name:",company['company']['name']," Company type:",company['company']['company_type'])
                    count+=1
            print("\n\n")
            

In [236]:
df_warchest.head(5).apply(get_company_names, axis=1)

https://api.opencorporates.com/v0.4/officers/search?q=BEN+CLARK+III&api_token=BSSwhXpP7xJ5couOddoY
https://api.opencorporates.com/v0.4/officers/search?q=JOE+NELSON&api_token=BSSwhXpP7xJ5couOddoY
Officer name: BILLY JOE NELSON
Officer address: 402 NORTH MOOSE ST.,
MORRILTON, AR 72110
Officer company name: DRILLING CLEANERS, INC. (R) 

Officer name: BILLY JOE NELSON
Officer address: 4266 EAST HIGHWAY 114, RHOME, TX, 76078
Officer company name: GLOBAL MACHINE & TOOL, LLC 

Officer name: BOBBY JOE NELSON
Officer address: 7512 TAYLOR AVE, EVANSVILLE, IN, 47715, USA
Officer company name: NELSON INSTALLATIONS, L.L.C. 

Officer name: BOBBY JOE NELSON
Officer address: 7377 REDWOOD BLVD STE B
NOVATO CA 94947
Officer company name: BEST CHOICE AUTO SALES, LLC 

Officer name: C JOE NELSON JR
Officer address: RT 2 BOX 754, ANAHUAC, TX, 77514
Officer company name: FISHERMAN'S HARVEST, INC. 

Officer name: Calloway, Joe Nelson
Officer address: 1716 HIGHWAY 52 N.
ALBEMARLE NC 28001
Officer company name

https://api.opencorporates.com/v0.4/officers/search?q=ANTHONY+LEWIS&api_token=BSSwhXpP7xJ5couOddoY
Officer name: ADAM WILLIAM LEWIS ANTHONY
Officer address: 41 SKYLINES BUSINESS VILLAGE/ FAO JSA PARTNERS ACC LIMEHARBOUR
CANARY WHARF, LONDON, E14 9TS, UNITED KINGDOM
Officer company name: MILE HIGH PARAGLIDING LIMITED 

Officer name: ALEXANDER ANTHONY LEWIS
Officer address: 31 HIGH STREET, STOCKBRIDGE, HAMPSHIRE, SO20 6EY, ENGLAND
Officer company name: THE BRIDGE FESTIVAL LTD 

Officer name: ALEXANDER LEWIS ANTHONY GILBERT
Officer address: UNIT E ASTON ROAD BUSINESS PARK ASTON ROAD
WEM, SHREWSBURY, SY4 5BA, UNITED KINGDOM
Officer company name: EMERALD TRADING WASTE SOLUTIONS LTD 

Officer name: ANDREW LEWIS ANTHONY BURNETT
Officer address: 71-75 SHELTON STREET
COVENT GARDEN, LONDON, WC2H 9JQ, UNITED KINGDOM
Officer company name: VIRTUAL WATER LIMITED 

Officer name: ANTHONY A LEWIS
Officer address: 1017 Fair winds circle, PLANT CITY, FL 33563
Officer company name: CORTEX ENTERPRISES LLC.

Date
2014-11-01    None
2014-11-01    None
2014-11-02    None
2014-11-03    None
2014-11-03    None
dtype: object

In [24]:
#I gathered from John and Peter that we are specifically interested in LLCs that give to Cuomo, so here they are.

df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)]

,Donor,Address,Filer ID,Check No.,Amount,Recipient,Transaction Code,Contribution Code,Contribution Type,Fiscal Report
Date,,,,,,,,,,
2014-11-03,"6970 SCHUYLER ROAD, LLC","25 DEBOER DRIVE GLEN ROCK, NJ, 07452",A31966,1017,2250.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,27 Day Post General
2014-11-03,170 WEST 75TH STREET RETAIL LLC,"170 WEST 75TH STREET NEW YORK, NY, 10023",A31966,10056,10000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,27 Day Post General
2014-11-03,"SINATRA & COMPANY REAL ESTATE, LLC","3350 DELAWARE AVENUE BUFFALO, NY, 14217",A31966,3173,5000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,27 Day Post General
2014-11-03,"WATERFRONT PROPERTY MANAGEMENT, LLC","320 ROEBLING STREET BROOKLYN, NY, 11211",A31966,12101,40000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,27 Day Post General
2014-11-03,SHARED CONCEPTS LLC,"677 BROADWAY ALBANY, NY, 12207",A31966,CC,25000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,27 Day Post General
2014-11-04,750 THIRD OWNER LLC,"420 LEXINGTON AVENUE NEW YORK, NY, 10170",A31966,NaN,5000.0,ANDREW CUOMO 2018 INC.,In-Kind Contributions,NaN,Property Given,27 Day Post General
2014-12-04,750 THIRD OWNER LLC,"420 LEXINGTON AVENUE NEW YORK, NY, 10170",A31966,NaN,10000.0,ANDREW CUOMO 2018 INC.,In-Kind Contributions,NaN,Property Given,Periodic Jan.
2014-12-11,NS OUEST LLC,"24 WEST 57TH STREET NEW YORK, NY, 10019",A31966,NaN,7750.0,ANDREW CUOMO 2018 INC.,In-Kind Contributions,NaN,Campaign Expenses Paid,Periodic Jan.
2015-03-18,TERN CONSTRUCTION & DEVELOPMENT LLC,"1777 ROUTE 6 CARMEL, NY, 10512",A31966,chk 323,15000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,Periodic July


In [30]:
#How many are there, what percent of the total donations are they??
rows, _ = df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)].shape
total_rows, _ = df_warchest.shape
print(rows, "or", round(rows/total_rows * 100, 2), "% of donations are from LLCs.")

550 or 10.49 % of donations are from LLCs.


In [31]:
df_llc = df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)]

In [36]:
print("The total given by LLCs is:", '${:,.2f}'.format(df_llc['Amount'].sum()))
print("Which is", round(df_llc['Amount'].sum()/df_warchest['Amount'].sum(), 2), "% of the total dollar amount.")

The total given by LLCs is: $6,610,176.00
Which is 0.22 % of the total dollar amount.


In [61]:
unique, _ = df_llc.groupby("Address").first().shape

print("There are", unique, "unique LLCs in the past 4 years of campaign contributions.")

There are 369 unique LLCs in the past 4 years of campaign contributions.


In [68]:
avg_donation = df_warchest['Amount'].mean()
llc_av_donation = df_llc['Amount'].mean()
print("Cuomo's donors give an average of", '${:,.2f}'.format(avg_donation), "while LLCs give an average of", '${:,.2f}'.format(llc_av_donation))

Cuomo's donors give an average of $5,735.81 while LLCs give an average of $12,018.50


In [57]:
#So, while LLCs are only 10% of the number of Cuomo's donations,
#they are more than twice that amount in terms of $.
#Who are they? 170 West 75th Street Retail doesn't tell us much.
#Let's see if their addresses match up with any registered addresses from OpenCorporates' data.
#I'm just going to query the api once for each unique address so I don't use up my precious quota.

company_names = {}
df_llc.groupby("Address").first().reset_index().apply(get_company_names, axis=1)

Original Address and Donor:  1 ASTELLAS WAY NORTHBROOK, IL, 60062 ASTELLAS US LLC POLITICAL ACTION COMMITTEE
Matched Registered Address 1
Company name: ASTELLAS INNOVATION MANAGEMENT LLC  Company type: Foreign Limited Liability Company (LLC)
Matched Registered Address 2
Company name: ASTELLAS INSTITUTE FOR REGENERATIVE MEDICINE  Company type: Foreign Corporation
Matched Registered Address 3
Company name: ASTELLAS INSTITUTE FOR REGENERATIVE MEDICINE  Company type: FOREIGN STOCK
Matched Registered Address 4
Company name: ASTELLAS PHARMA GLOBAL DEVELOPMENT, INC.  Company type: Corporation - Foreign - Profit
Matched Registered Address 5
Company name: ASTELLAS PHARMA GLOBAL DEVELOPMENT, INC.  Company type: Stock
Matched Registered Address 6
Company name: ASTELLAS PHARMA GLOBAL DEVELOPMENT, INC.  Company type: FOREIGN STOCK
Matched Registered Address 7
Company name: ASTELLAS PHARMA US, INC.  Company type: Foreign Profit Corporation
Matched Registered Address 8
Company name: ASTELLAS PHARMA U

Original Address and Donor:  1155 AVENUE OF THE AMERICAS NEW YORK, NY, 10036 DOLP 1155 PROPERTIES LLC
Matched Registered Address 1
Company name: ADVANCED CORPORATE SOLUTIONS LTD.  Company type: Foreign Corporation
Matched Registered Address 2
Company name: ADVANCED CORPORATE SOLUTIONS LTD.  Company type: Foreign Corporate
Matched Registered Address 3
Company name: ADVANCED CORPORATE SOLUTIONS LTD.  Company type: FOREIGN STOCK
Matched Registered Address 4
Company name: ADVANCED CORPORATE SOLUTIONS LTD.  Company type: Foreign Profit Corporation
Matched Registered Address 5
Company name: AKRITIV TECHNOLOGIES, INC.  Company type: FOREIGN STOCK
Matched Registered Address 6
Company name: GENPACT COLLECTIONS LLC  Company type: FOREIGN LIMITED LIABILITY COMPANY
Matched Registered Address 7
Company name: GENPACT COLLECTIONS LLC  Company type: Limited Liability Company
Matched Registered Address 8
Company name: GENPACT COLLECTIONS LLC  Company type: Collection Agency - Foreign
Matched Registered

Original Address and Donor:  132 NASSAU ST NEW YORK, NY, 10038 THE PARKSIDE GROUP LLC
Matched Registered Address 1
Company name: INOVA SOFTWARE INC.  Company type: Stock



Original Address and Donor:  13401 20TH AVE COLLEGE POINT, NY, 11356 MATTONE GROUP MANAGEMENT LLC
Matched Registered Address 1
Company name: Arb (Lebanon) Realty (ISM) LLC  Company type: Limited Liability Company
Matched Registered Address 2
Company name: Arb (Lebanon) Realty (JP) LLC  Company type: Limited Liability Company



Original Address and Donor:  1600 NW 163RD ST MIAMI, FL, 33169 BOLINGBROOK PROPERTY PARTNERS LLC
Matched Registered Address 1
Company name: Hannah and Dunn, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 2
Company name: PACIFIC WINES AND SPIRITS, LLC  Company type: DOMESTIC
Matched Registered Address 3
Company name: PHOENIX WINE & SPIRITS, LLC  Company type: LLC - Domestic
Matched Registered Address 4
Company name: SOUTHERN GLAZER'S DISTRIBUTORS OF IOWA, LLC  

Original Address and Donor:  200 LIBERTY ST NEW YORK, NY, 10281 BP ONE WFC CO. LLC
Matched Registered Address 1
Company name: ADVISOR GROUP, INC.  Company type: Stock
Matched Registered Address 2
Company name: ADVISOR GROUP, INC.  Company type: Foreign For-Profit Corporation
Matched Registered Address 3
Company name: ADVISOR GROUP, INC.  Company type: Foreign Profit Corporation
Matched Registered Address 4
Company name: ADVISOR GROUP, INC.  Company type: Corporation - Foreign - Profit
Matched Registered Address 5
Company name: ADVISOR GROUP, INC.  Company type: CORPORATION, GENERAL BUSINESS
Matched Registered Address 6
Company name: ASSOCIATED PRESS (THE)  Company type: CORPORATION, GENERAL NON PROFIT
Matched Registered Address 7
Company name: AZA INT CORPORATION  Company type: Foreign Corporation
Matched Registered Address 8
Company name: Advisor Group, Inc.  Company type: Business Corporation
Matched Registered Address 9
Company name: Advisor Group, Inc.  Company type: For-profit Cor

Original Address and Donor:  300 GLEED AVE EAST AURORA, NY, 14052 ABSOLUT FACILITIES MANAGEMENT, LLC



Original Address and Donor:  302 WASHINGTON AVENUE EXT ALBANY, NY, 12203 MT. IDA APARTMENTS, LLC
Matched Registered Address 1
Company name: BBL CARLTON, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 2
Company name: BBL CONSTRUCTION SERVICES, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 3
Company name: BBL Construction Services, LLC  Company type: Limited Liability Company



Original Address and Donor:  320 N JENSEN RD VESTAL, NY, 13850 JFM REALTY LLC
Matched Registered Address 1
Company name: EAGLE ENTERTAINMENT GROUP INC.  Company type: Foreign Corporation



Original Address and Donor:  320 ROEBLING ST BROOKLYN, NY, 11211 WATERFRONT PROPERTY MANAGEMENT, LLC
Matched Registered Address 1
Company name: COMFORT BEDDING OF THE USA, LLC  Company type: Foreign Limited Liability Company



Original Address and Donor:  32

...

Original Address and Donor:  500 N BROADWAY JERICHO, NY, 11753 HAGGERTY MUNZ PLLC
Matched Registered Address 1
Company name: BRISTOL FINANCIAL SERVICES, INC.  Company type: Foreign Corporation
Matched Registered Address 2
Company name: BRISTOL FINANCIAL SERVICES, INC.  Company type: Foreign Corporate
Matched Registered Address 3
Company name: ELIAS PROPERTIES SALEM CROSSING, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 4
Company name: ELIAS PROPERTIES VIRGINIA BEACH CENTERVILLE, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 5
Company name: ELIAS PROPERTIES VIRGINIA BEACH LYNNHAVEN, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 6
Company name: Elias Properties Virginia Beach Independence, LLC  Company type: Foreign Limited Liability Company



Original Address and Donor:  5309 TRANSPORTATION BLVD CLEVELAND, OH, 44125 THE NRP GROUP LLC
Matched Registered Address 1
Company name: AVON SEN

Original Address and Donor:  75 THRUWAY PARK DR WEST HENRIETTA, NY, 14586 KONAR ENTERPRISES, LLC
Matched Registered Address 1
Company name: CAREER SYSTEMS DEVELOPMENT CORPORATION  Company type: FOREIGN PROFIT
Matched Registered Address 2
Company name: LECESSE CONSTRUCTION SERVICES LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 3
Company name: LECESSE CONSTRUCTION SERVICES LLC  Company type: Foreign Limited Liability Company



Original Address and Donor:  80 STATE ST ALBANY, NY, 12207 US IMMIGRATION FUND - NY LLC
Matched Registered Address 1
Company name: ASTHMA & ALLERGY FOUNDATION OF AMERICA  Company type: Foreign Nonprofit Corporation
Matched Registered Address 2
Company name: BERLITZ LANGUAGES, INC.  Company type: Foreign Profit Corporation
Matched Registered Address 3
Company name: BR HOLDINGS ASSOCIATES INC.  Company type: Stock
Matched Registered Address 4
Company name: CLARIANT CORPORATION  Company type: Foreign Profit Corporation
Matched Regist

Original Address and Donor:  PO BOX 7033 INDIANAPOLIS, IN, 46207 OYSTER BAY DEVELOPMENT, LLC
Matched Registered Address 1
Company name: BRIGHTWOOD PLAZA INC  Company type: Domestic For-Profit Corporation
Matched Registered Address 2
Company name: CASTLETON FRESH, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 3
Company name: COBBLESTONE COURT LIMITED PARTNERSHIP  Company type: Domestic Limited Partnership
Matched Registered Address 4
Company name: CRYSTAL LAKE SHOPPING CENTER LIMITED PARTNERSHIP  Company type: Domestic Limited Partnership
Matched Registered Address 5
Company name: DCT INVESTORS, LLC  Company type: Domestic Limited Liability Company
Matched Registered Address 6
Company name: Fashion Centre Mall, LLC  Company type: Foreign Limited Liability Company
Matched Registered Address 7
Company name: GILROY PREMIUM OUTLETS, LLC  Company type: FOREIGN
Matched Registered Address 8
Company name: GREENWOOD PARK MALL, LLC  Company type: Foreign Limited 

Original Address and Donor:  STE 894, 199 LEE AVE BROOKLYN, NY, 11211 JL N ASSOCIATES LLC
Matched Registered Address 1
Company name: THE HERALD SQUARE, LLC  Company type: DOMESTIC LIMITED LIABILITY COMPANY



Original Address and Donor:  STE 900, 110 W FAYETTE ST SYRACUSE, NY, 13202 BOUSQUET HOLSTEIN PLLC
Matched Registered Address 1
Company name: FRIENDS OF THE SARAH MCLACHLAN SCHOOL OF MUSIC SOCIETY, INC.  Company type: FOREIGN NONPROFIT



Original Address and Donor:  THE CLINTON EXCHANGE, 4 CLINTON SQUARE SYRACUSE, NY, 13202 DEMBO DEVELOPMENT ASSOCIATES LLC
Matched Registered Address 1
Company name: HOLYOKE MALL COMPANY LIMITED PARTNERSHIP  Company type: Foreign Limited Partnership (LP)
Matched Registered Address 2
Company name: INDEPENDENCE CENTER NEWCO LLC  Company type: Foreign Limited Liability Company (LLC)
Matched Registered Address 3
Company name: PCH CENTER HOLDINGS INC.  Company type: Foreign Corporation
Matched Registered Address 4
Company name: PYRAMID MANAGEMENT GROUP, 

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
339    None
340    None
341    None
342    None
343    None
344    None
345    None
346    None
347    None
348    None
349    None
350    None
351    None
352    None
353    None
354    None
355    None
356    None
357    None
358    None
359    None
360    None
361    None
362    None
363    None
364    None
365    None
366    None
367    None
368    None
Length: 369, dtype: object

There are 369 unique LLCs in the past 4 years of campaign contributions.
